In [1358]:
import random
import copy 

#### Implementacja krzyżowania jedno, dwu i trzy punktowego oraz krzyżowania jednorodnego
- `x1` - chromosom pierwszy, lista
- `x2` - chromosom drugi, lista
- `crossoverProb` - prawdopodobieństwo krzyżowania, liczba z zakresu 0,1

In [756]:
def singlePointCrossover(x1, x2):
    point = random.randrange(1, len(x1))
    temp = x1[:point]
    x1[:point] = x2[:point]
    x2[:point] = temp
    return x1, x2

In [1108]:
def twoPointsCrossover(x1, x2):
    points = random.sample(range(1, len(x1)), 2)
    point1 = min(points)
    point2 = max(points)
    temp = x1[point1:point2]
    x1[point1:point2] = x2[point1:point2]
    x2[point1:point2] = temp
    return x1, x2

In [787]:
def threePointsCrossover(x1, x2):
    points = random.sample(range(1, len(x1)), 3)
    point1 = min(points)
    point2 = max(points)
    point3 = [i for i in points if i != point1 and i != point2][0]

    temp = x1[point1:point2]
    x1[point1:point2] = x2[point1:point2]
    x2[point1:point2] = temp
    temp = x1[point3:]
    x1[point3:] = x2[point3:]
    x2[point3:] = temp
    
    return x1, x2

In [1113]:
def probUniformCrossover(x1, x2, crossoverProb):
    for index in range(len(x1)):
        prob = random.random()
        if prob < crossoverProb:
            temp = x1[index]
            x1[index] = x2[index]
            x2[index] = temp
    return x1, x2

### Implementacja krzyżowania jedno, dwu i trzy punktowego oraz krzyżowania jednorodnego obiektowo
- `ind1` - pierwszy osobnik, obiekt typu Individual
- `ind2` - pierwszy osobnik, obiekt typu Individual
- `crossoverProb` - prawdopodobieństwo krzyżowania, liczba z zakresu 0,1

In [1105]:
def singlePointCrossoverObj(ind1, ind2):
    x1, x2 = singlePointCrossover(ind1.X.binary, ind2.X.binary)
    ind1.setX(x1)
    ind2.setX(x2)

    y1, y2 = singlePointCrossover(ind1.Y.binary, ind2.Y.binary)
    ind1.setY(y1)
    ind2.setY(y2)

In [1109]:
def twoPointsCrossoverObj(ind1, ind2):
    x1, x2 = twoPointsCrossover(ind1.X.binary, ind2.X.binary)
    ind1.setX(x1)
    ind2.setX(x2)

    y1, y2 = twoPointsCrossover(ind1.Y.binary, ind2.Y.binary)
    ind1.setY(y1)
    ind2.setY(y2)

In [1110]:
def threePointsCrossoverObj(ind1, ind2):
    x1, x2 = threePointsCrossover(ind1.X.binary, ind2.X.binary)
    ind1.setX(x1)
    ind2.setX(x2)

    y1, y2 = threePointsCrossover(ind1.Y.binary, ind2.Y.binary)
    ind1.setY(y1)
    ind2.setY(y2)

In [1141]:
def probUniformCrossoverObj(ind1, ind2, crossoverProb):
    x1, x2 = probUniformCrossover(ind1.X.binary, ind2.X.binary, crossoverProb)
    ind1.setX(x1)
    ind2.setX(x2)

    y1, y2 = probUniformCrossover(ind1.Y.binary, ind2.Y.binary, crossoverProb)
    ind1.setY(y1)
    ind2.setY(y2)

## Implementacja chromosomu
- `bits` - liczba bitów chromosomu, liczba całkowita
- `binary` - chromosom w reprezentacji binarnej, tablica jednowymiarowa `ndarray`
- `decimal` - chromosom w reprezentacji dziesiętnej, liczba całkowita

#### setBinary - implementacja binarnej reprezentacji chromosomu + konfiguracja dokładności
- `bits` - liczba bitów chromosomu, liczba całkowita

#### setDecimal - implementacja rozkodowania chromosomu - konwersji  z reprezentacji binarnej do reprezentacji dziesiętnej
- `binary` - chromosom w reprezentacji binarnej, tablica jednowymiarowa `ndarray`
- `decimal` - chromosom w reprezentacji dziesiętnej, liczba całkowita

#### onePointMutation, twoPointsMutation, edgeMutation -  implementacja mutacji jedno i dwupunktowej, brzegowej
- `mutationProb` - prawdopodobieństwo mutacji, liczba z zakresu 0,1

#### inversion - implementacja operatora inwersji
- `inversionProb` - prawdopodobieństwo inwersji, liczba z zakresu 0,1

In [1133]:
class Chromosome:
    def __init__(self, bits):
        self.bits = bits
        self.binary = self.setBinary()
        self.decimal = self.setDecimal()
        
    def setBinary(self):
        return [random.randrange(0, 2) for row in range(self.bits)]
    
    def setDecimal(self):
        decimal = 0
        position = len(self.binary)
        for bit in self.binary:
            position -= 1
            decimal += bit * pow(2, position)
        return decimal
    
    def oppositeBit(bit):
        return 1 if bit == 0 else 0
    
    def onePointMutation(self, mutationProb):
        prob = random.random()
        if prob < mutationProb:
            point = random.randrange(0, self.bits)
            self.binary[point] = oppositeBit(self.binary[point])
        self.decimal = self.setDecimal()
        
    def twoPointsMutation(self, mutationProb):
        points = random.sample(range(0, self.bits), 2)
        for point in points:
            prob = random.random()
            if prob < mutationProb:
                self.binary[point] = oppositeBit(self.binary[point])
        self.decimal = self.setDecimal()
        
    def edgeMutation(self, mutationProb):
        points = [0, self.bits - 1]
        for point in points:
            prob = random.random()
            if prob < mutationProb:
                self.binary[point] = oppositeBit(self.binary[point])
        self.decimal = self.setDecimal()
        
    def inversion(self, inversionProb):
        prob = random.random()
        if prob < inversionProb:
            disables = True
            while(disables):
                points = random.sample(range(1, self.bits), 2)
                point1 = min(points)
                point2 = max(points)
                disables = False if (point2 >= point1 + 2) else True
            inversion = self.binary[point1:point2]
            inversion.reverse()
            self.binary = self.binary[:point1] + inversion + self.binary[point2:] 
        self.decimal = self.setDecimal()

## Implementacja osobnika o dwóch chromosomach
- `bits` - liczba bitów chromosomu, liczba całkowita
- `X` - chromosom X osobnika, obiekt typu Chromosome
- `Y` - chromosom Y osobnika, obiekt typu Chromosome

#### boothFunc - implementacja wartości funkcji Booth osobnika
#### reciprocalBoothFunc - implementacja odwotności wartości funkcji Booth osobnika

In [1295]:
class Individual:
    def __init__(self, bits):
        self.bits = bits
        self.X = self.setChromosome()
        self.Y = self.setChromosome()
        
    def setChromosome(self):
        return Chromosome(self.bits)
    
    def setX(self, binary):
        self.X.binary = binary
        self.X.decimal = self.X.setDecimal()
        
    def setY(self, binary):
        self.Y.binary = binary
        self.Y.decimal = self.Y.setDecimal()
    
    def boothFunc(self):
        return pow((self.X.decimal + 2 * self.Y.decimal - 7), 2) + pow((2 * self.X.decimal + self.Y.decimal - 5), 2)
    
    def reciprocalBoothFunc(self):
        return 1 / self.boothFunc()

## Implementacja populacji
- `chromoBits` - liczba bitów chromosomów osobników populacji, liczba całkowita
- `size` - liczba osobnikow w populacji, liczba całkowita

#### bestSelection - implementacja selekcji najlepszych
- `percent` - procent najlepszych osobników, liczba całkowita
- `newPopulation` - najlepiej przystostowane osobniki z populacji, nowa populacja osobników

#### tournamentSelection - implementacja selekcji turniejowej
- `k` -  wielkość turnieju, liczba całkowita
- `pop.individuals[0]` - osobnik wybrany z populacji selekcją turniejową, obiekt klasy Individual

#### rouletteWheelSelection - implementacja selekcji kołem ruletki
- `self.individuals[bestIndex]` - osobnik wybrany z populacji selekcją kołem ruletki, obiekt klasy Individual

##### Dodatkowo: calculateDistribution - implementacja wyznaczenia dystrybucji osobników dla danej populacji
   - `temp` - dystrybucje osobników z populacji, lista


In [1339]:
class Population:
    def __init__(self, size, chromoBits):
        self.size = size
        self.chromoBits = chromoBits
        if size > 0:
            self.individuals = self.setPopulation()
        else :
            self.individuals = []
        
    def setPopulation(self):    
        return [Individual(self.chromoBits) for row in range(self.size)]
    
    def addIndividual(self, ind):    
        self.individuals.append(ind)
        self.size += 1
        
    def bestSelection(self, percent = 30):
        popAdaptation = []
        bestSelected = []
        N = int(self.size * percent / 100)

        for ind in self.individuals:
            popAdaptation.append(ind.boothFunc())  

        for i in range(0, N):  
            best = max(popAdaptation)
            bestSelected.append(best)
            popAdaptation.remove(best) 

        newPopulation = Population(0, 8)
        for best in bestSelected:
            for ind in self.individuals:
                if ind.boothFunc() == best:
                    newPopulation.addIndividual(ind)
        return newPopulation
    
    
    def tournamentSelection(self, k):
        pop = copy.deepcopy(self)    
        popSize = len(pop.individuals)

        if popSize == 1:
            return pop.individuals[0]
        else:
            best = Population(0, self.chromoBits)
            # create tournaments using all individuals from population
            while(popSize > 0):
                popSize = len(pop.individuals)
                tournament = Population(0, self.chromoBits)

                # select k random individuals from population to new tournament (without repetition)
                if popSize >= k:
                    for i in range(k):
                        randInd = random.choice(pop.individuals)
                        tournament.addIndividual(randInd)
                        pop.individuals.remove(randInd)
                elif popSize > 1:
                    tournament = copy.deepcopy(pop)    
                    pop.individuals.clear() 
                else:
                    break

                # select the best individuals tournament (stochastic tournament selection)
                best.addIndividual(random.choice(tournament.individuals)) 

            return tournamentSelection(best, k)
        
    def calculateDistribution(self):
        temp = [] 
        # adaptation function
        for ind in self.individuals:
            temp.append(ind.reciprocalBoothFunc())
        adaptationSum = sum(temp)
        # calculate every individual probability
        temp = [adaptation/adaptationSum for adaptation in temp] 
        # calculate every individual probability
        for i in range(1, len(temp)):
            temp[i] += temp[i-1]
        return temp
    
    
    def rouletteWheelSelection(self):
        populationDist = self.calculateDistribution()
        self.showRouletteWheel(populationDist)
        # spin roulette wheel and find the best individual
        rand = random.random()
        previous = 0
        bestIndex = -1
        for index, value in enumerate(populationDist):
            if(previous <= rand <= value):
                bestIndex = index
            previous = value
        return self.individuals[bestIndex], fig

##### showRouletteWheel - implementacja prezentacji koła ruletki dla populacji
   - `individuals` - lista osobników, lista obiektów typu Individual
   - `populationDist` - dystrybucje osobników, lista 

In [1355]:
import plotly.graph_objects as go

def showRouletteWheel(individuals, populationDist):
    stringPop = []
    for ind in individuals:
        stringPop.append(str(ind.X.binary) + str(ind.Y.binary))
    fig = go.Figure(data = [go.Pie(labels = stringPop, values = populationDist)])
    fig.show()

In [1356]:
pop = Population(20,8)
dist = pop.calculateDistribution()
showRouletteWheel(pop.individuals, dist)